In [24]:
import pandas as pd
import re

In [6]:
orders = pd.read_csv('./data/Webull_Orders_Records.csv')
options = pd.read_csv('./data/Webull_Orders_Records_Options.csv')

In [9]:
options['Placed Time'] = pd.to_datetime(options['Placed Time'], infer_datetime_format=True)
options['Filled Time'] = pd.to_datetime(options['Filled Time'], infer_datetime_format=True)

In [11]:
options['Wait'] = options['Filled Time'] - options['Placed Time']

In [15]:
filled_filter = options['Status'] == 'Filled'
options = options[filled_filter]

In [16]:
options

,Name,Symbol,Side,Status,Filled,Total Qty,Price,Avg Price,Time-in-Force,Placed Time,Filled Time,Wait
0,LCID 01/14/2022 00:00:00 EST Call $49.00,LCID220114C00049000,Sell,Filled,12,12,@0.580,0.58,DAY,2022-01-11 11:13:31,2022-01-11 12:53:48,01:40:17
1,LCID 01/14/2022 00:00:00 EST Call $49.00,LCID220114C00049000,Buy,Filled,12,12,@0.520,0.52,DAY,2022-01-11 11:03:17,2022-01-11 11:05:03,00:01:46
2,SPY 01/12/2022 00:00:00 EST Call $465.00,SPY220112C00465000,Sell,Filled,7,7,@0.900,0.90,DAY,2022-01-10 13:08:43,2022-01-10 13:08:45,00:00:02
4,SPY 01/12/2022 00:00:00 EST Call $465.00,SPY220112C00465000,Buy,Filled,7,7,@0.750,0.75,DAY,2022-01-10 11:50:58,2022-01-10 11:51:17,00:00:19
6,AMC 01/07/2022 00:00:00 EST Put $23.00,AMC220107P00023000,Sell,Filled,9,9,@0.550,0.55,DAY,2022-01-07 10:16:00,2022-01-07 10:21:48,00:05:48
...,...,...,...,...,...,...,...,...,...,...,...,...
130,Z 06/18/2021 00:00:00 EDT Call $120.00,Z 06/18/2021 00:00:00 EDT Call $120.00,Sell,Filled,2,2,@0.610,0.61,DAY,2021-06-14 15:54:24,2021-06-14 15:54:24,00:00:00
131,IVR 06/18/2021 00:00:00 EDT Call $5.00,IVR 06/18/2021 00:00:00 EDT Call $5.00,Buy,Filled,3,3,@0.170,0.17,DAY,2021-06-14 11:11:17,2021-06-14 11:19:17,00:08:00
133,BLUE 07/16/2021 00:00:00 EDT Call $45.00,BLUE 07/16/2021 00:00:00 EDT Call $45.00,Buy,Filled,1,1,@1.35,1.35,DAY,2021-06-14 10:21:02,2021-06-14 10:23:27,00:02:25
134,Z 06/18/2021 00:00:00 EDT Call $120.00,Z 06/18/2021 00:00:00 EDT Call $120.00,Buy,Filled,1,1,@0.930,0.93,DAY,2021-06-14 09:51:18,2021-06-14 09:52:46,00:01:28


In [57]:
options['Symbol Price'] = options['Name'].apply(lambda x: re.match(r"(.+)(\$.*)$", x).group(2))
options['Symbol Price'] = options['Symbol Price'].astype(str).str[1:]
options['Symbol Price'] = options['Symbol Price'].astype(float)
options['Symbol'] = options['Symbol'].astype(str).str[0:5].replace('[^a-zA-Z]', '')
options['Symbol'] = options['Symbol'].astype(str).str.rstrip('0123456789')
cols = [
    'Symbol', 'Symbol Price', 'Side', 'Status', 'Filled', 'Total Qty', 'Price',
    'Avg Price', 'Time-in-Force', 'Placed Time', 'Filled Time', 'Wait', 'Name'
       ]
options = options[cols]

In [58]:
options

,Symbol,Symbol Price,Side,Status,Filled,Total Qty,Price,Avg Price,Time-in-Force,Placed Time,Filled Time,Wait,Name
0,LCID,49.0,Sell,Filled,12,12,@0.580,0.58,DAY,2022-01-11 11:13:31,2022-01-11 12:53:48,01:40:17,LCID 01/14/2022 00:00:00 EST Call $49.00
1,LCID,49.0,Buy,Filled,12,12,@0.520,0.52,DAY,2022-01-11 11:03:17,2022-01-11 11:05:03,00:01:46,LCID 01/14/2022 00:00:00 EST Call $49.00
2,SPY,465.0,Sell,Filled,7,7,@0.900,0.90,DAY,2022-01-10 13:08:43,2022-01-10 13:08:45,00:00:02,SPY 01/12/2022 00:00:00 EST Call $465.00
4,SPY,465.0,Buy,Filled,7,7,@0.750,0.75,DAY,2022-01-10 11:50:58,2022-01-10 11:51:17,00:00:19,SPY 01/12/2022 00:00:00 EST Call $465.00
6,AMC,23.0,Sell,Filled,9,9,@0.550,0.55,DAY,2022-01-07 10:16:00,2022-01-07 10:21:48,00:05:48,AMC 01/07/2022 00:00:00 EST Put $23.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,Z 06/,120.0,Sell,Filled,2,2,@0.610,0.61,DAY,2021-06-14 15:54:24,2021-06-14 15:54:24,00:00:00,Z 06/18/2021 00:00:00 EDT Call $120.00
131,IVR,5.0,Buy,Filled,3,3,@0.170,0.17,DAY,2021-06-14 11:11:17,2021-06-14 11:19:17,00:08:00,IVR 06/18/2021 00:00:00 EDT Call $5.00
133,BLUE,45.0,Buy,Filled,1,1,@1.35,1.35,DAY,2021-06-14 10:21:02,2021-06-14 10:23:27,00:02:25,BLUE 07/16/2021 00:00:00 EDT Call $45.00
134,Z 06/,120.0,Buy,Filled,1,1,@0.930,0.93,DAY,2021-06-14 09:51:18,2021-06-14 09:52:46,00:01:28,Z 06/18/2021 00:00:00 EDT Call $120.00


In [66]:
side_counts = options['Side'].value_counts()
symbol_counts = options['Symbol'].value_counts()
average_price = options.groupby('Symbol')['Symbol Price'].mean()

In [64]:
average_price

Symbol
AAPL     152.500000
AMC       23.000000
AMD      143.000000
BLNK      35.000000
BLUE      45.000000
CVS       93.000000
EAR       10.000000
EBAY      78.000000
FCEL       7.000000
FDX      247.500000
GLOP       7.500000
INTC      62.000000
ITUB       7.000000
IVR        4.666667
LCID      42.000000
MCD      265.000000
MSFT     325.000000
NVDA     290.000000
SDC       10.000000
SPY      470.000000
SPY      475.000000
TGT      260.000000
TMC       20.000000
TR        40.000000
WBA       50.000000
WFC       54.000000
WKEY      10.000000
WM       160.000000
Z 06/    120.000000
ZEV       15.000000
Name: Symbol Price, dtype: float64

In [61]:
symbol_counts

AMD                                                              6
SPY                                                              4
LCID                                                             4
AAPL                                                             4
TR                                                               3
FCEL                                                             3
SPY                                                              3
Z 06/                                                            3
MCD                                                              3
IVR                                                              3
CVS                                                              3
TMC                                                              3
WKEY                                                             2
AMC                                                              2
WFC                                                           